### Data Engineering

In [ ]:
#import libraries
import os
import pandas as pd
import numpy as np
import copy
import random
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(32)
np.random.seed(32)
random.seed(32)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# dataFolder= '/content/drive/MyDrive/ECE1508/Code'
dataFolder= '.'
os.chdir(dataFolder)
print("Current Working Directory: ", os.getcwd())

Current Working Directory:  /content/drive/.shortcut-targets-by-id/1FsmqwEiZJAaz0p8i6_y9fb9O2BZref7v/ECE1508/Code


In [ ]:
#load dataset
csvPath = "./data/btcusd_1-min_data.csv"

df = pd.read_csv(csvPath)

#convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s', utc=True)
df.set_index('Timestamp', inplace=True)

#sort by datetime index
df.sort_index(inplace=True)

#print first 5 rows
df.head()


,Open,High,Low,Close,Volume
Timestamp,,,,,
2012-01-01 10:01:00+00:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:02:00+00:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:03:00+00:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:04:00+00:00,4.58,4.58,4.58,4.58,0.0
2012-01-01 10:05:00+00:00,4.58,4.58,4.58,4.58,0.0


In [ ]:
# print last 5 rows
df.tail()

,Open,High,Low,Close,Volume
Timestamp,,,,,
2025-11-18 23:53:00+00:00,92836.0,92850.0,92819.0,92835.0,38.001801
2025-11-18 23:54:00+00:00,92846.0,92846.0,92826.0,92827.0,0.919689
2025-11-18 23:55:00+00:00,92825.0,92825.0,92812.0,92812.0,0.193107
2025-11-18 23:56:00+00:00,92812.0,92857.0,92812.0,92857.0,0.324576
2025-11-18 23:57:00+00:00,92886.0,92949.0,92886.0,92949.0,0.404128


Perform Raw Data Analysis

In [ ]:
#check for missing values
missingValues = df.isnull().sum()
print("Missing Values in each column:\n", missingValues)

missingValues = df.isna().sum()
print("Missing Values in each column:\n", missingValues)

Missing Values in each column:
 Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64
Missing Values in each column:
 Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [ ]:
#check for duplicate rows by seeing if any timestamps are repeated
duplicateRows = df.index.duplicated().sum()
print("Number of duplicate rows based on Timestamp index:", duplicateRows)

Number of duplicate rows based on Timestamp index: 0


In [ ]:
#remove outliers using interquartile range
Q1 =df['Close'].quantile(0.25)
Q3 = df['Close'].quantile(0.75)
IQR = Q3 - Q1
lowerBound = Q1 - 1.5 * IQR
upperBound= Q3 + 1.5 * IQR

#print number of outliers
outliers =df[(df['Close']< lowerBound) | (df['Close'] > upperBound)]
print("Number of outliers in 'Close' column:", outliers.shape[0])

df =  df[(df['Close'] >= lowerBound) & (df['Close'] <= upperBound)]

#print number of outliers after removal
outliers = df[(df['Close'] < lowerBound) | (df['Close'] > upperBound)]
print("Number of outliers in 'Close' column after removal: ", outliers.shape[0])


Number of outliers in 'Close' column: 540932
Number of outliers in 'Close' column after removal:  0


In [ ]:
#check missing values again
missingValues = df.isna().sum()
print("Missing Values in each column:\n", missingValues)

Missing Values in each column:
 Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [ ]:
#resample data with 1 day frequency
dfDaily = df.resample('D').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

#print first 5 rows of daily resampled data
dfDaily.head()

,Open,High,Low,Close,Volume
Timestamp,,,,,
2012-01-01 00:00:00+00:00,4.58,4.84,4.58,4.84,10.000000
2012-01-02 00:00:00+00:00,4.84,5.00,4.84,5.00,10.100000
2012-01-03 00:00:00+00:00,5.00,5.32,5.00,5.29,107.085281
2012-01-04 00:00:00+00:00,5.29,5.57,4.93,5.57,107.233260
2012-01-05 00:00:00+00:00,5.57,6.46,5.57,6.42,70.328742


In [ ]:
#print last 5 rows of daily resampled data
dfDaily.tail()

,Open,High,Low,Close,Volume
Timestamp,,,,,
2025-04-05 00:00:00+00:00,NaN,NaN,NaN,NaN,0.000000
2025-04-06 00:00:00+00:00,NaN,NaN,NaN,NaN,0.000000
2025-04-07 00:00:00+00:00,75543.0,75543.0,74434.0,75405.0,509.787509
2025-04-08 00:00:00+00:00,NaN,NaN,NaN,NaN,0.000000
2025-04-09 00:00:00+00:00,75570.0,75740.0,74588.0,75364.0,251.734813


In [ ]:
#check for missing values in daily data
missingValues = dfDaily.isna().sum()
print("Missing Values in each column:\n", missingValues)

#drop rows with NaN values produced by resampling
dfDaily = dfDaily.dropna(how='any')

#check for missing values again
missingValues = dfDaily.isna().sum()
print("Missing Values in each column after dropping NaN values:\n",
      missingValues)

#print number of rows after dropping NaN values
print("Number of rows after dropping NaN values:", dfDaily.shape[0])

Missing Values in each column:
 Open      151
High      151
Low       151
Close     151
Volume      0
dtype: int64
Missing Values in each column after dropping NaN values:
 Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64
Number of rows after dropping NaN values: 4697


In [ ]:
#check if dataframe is continuous without missing timestamps
dfDaily['TimeDiff'] = dfDaily.index.to_series().diff().dt.total_seconds()
missingTimestamps = dfDaily[dfDaily['TimeDiff']!=86400].index
print("Number of missing timestamps:",len(missingTimestamps))

#show missing timestamps
print(missingTimestamps)

#compute number of missing daily rows (exclude first NaN)
timeDifferences =dfDaily['TimeDiff']
gaps = timeDifferences[timeDifferences.notna() & (timeDifferences!=86400)]

#determine missing rows per gap
missingPerGap = (gaps/86400).astype(int) - 1
# numMissingRows
numMissingRows = missingPerGap.sum()
print("Total number of missing daily rows:", numMissingRows)

#create a complete date range from start to end date
fullDateRange = pd.date_range(start=dfDaily.index.min(),
                              end=dfDaily.index.max(),
                              freq='D')

#reindex dataframe and add missing rows with NaN values
dfDaily = dfDaily.reindex(fullDateRange)

#interpolate missing daily rows
dfDaily[['Open', 'High','Low', 'Close','Volume']] = dfDaily[['Open',
                                                             'High',
                                                             'Low',
                                                             'Close',
                                                             'Volume'
                                                             ]].interpolate(
                                                                 method='linear'
                                                                 )
dfDaily = dfDaily.ffill()

#recompute total number of missing rows after interpolation
dfDaily['TimeDiff'] = dfDaily.index.to_series().diff().dt.total_seconds()
missingTimestamps=dfDaily[dfDaily['TimeDiff']!=86400].index
print("Number of missing timestamps after interpolation:",
      len(missingTimestamps))

Number of missing timestamps: 3
DatetimeIndex(['2012-01-01 00:00:00+00:00', '2025-04-07 00:00:00+00:00',
               '2025-04-09 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Timestamp', freq=None)
Total number of missing daily rows: 151
Number of missing timestamps after interpolation: 1


In [ ]:
dfDaily.head()

,Open,High,Low,Close,Volume,TimeDiff
2012-01-01 00:00:00+00:00,4.58,4.84,4.58,4.84,10.000000,NaN
2012-01-02 00:00:00+00:00,4.84,5.00,4.84,5.00,10.100000,86400.0
2012-01-03 00:00:00+00:00,5.00,5.32,5.00,5.29,107.085281,86400.0
2012-01-04 00:00:00+00:00,5.29,5.57,4.93,5.57,107.233260,86400.0
2012-01-05 00:00:00+00:00,5.57,6.46,5.57,6.42,70.328742,86400.0


First row missing time stamp is expected because no previous timestamp is available to compute a difference. However, we need interpolation for the other.

### Feature Engineering

In [ ]:
#add returns with formula of (current close - previous close) / previous close
dfDaily['Return'] = (
    dfDaily['Close']- dfDaily['Close'].shift(1)
    ) / dfDaily['Close'].shift(1)

#add moving averages
dfDaily['Moving_Average_7'] = dfDaily['Close'].rolling(
    window=7, min_periods=1).mean()
dfDaily['Moving_Average_30'] = dfDaily['Close'].rolling(
    window=30, min_periods=1).mean()

#add volatility as rolling standard deviation of close
dfDaily['Volatility_7'] = dfDaily['Close'].rolling(
    window=7, min_periods=1).std()
dfDaily['Volatility_30'] = dfDaily['Close'].rolling(
    window=30, min_periods=1).std()


#add cumulative returns
dfDaily['Cumulative_Return'] =(1 + dfDaily['Return']).cumprod() - 1

#add relative strength index (RSI) over 10-day period
deltaClose = dfDaily['Close'].diff()
gainPositive = deltaClose.where(deltaClose > 0, 0)
lossNegative = -deltaClose.where(deltaClose < 0, 0)
avgGain = gainPositive.rolling(window=10, min_periods=1).mean()
avgLoss = lossNegative.rolling(window=10, min_periods=1).mean()

#prevent division by zero and compute RSI
avgLoss = avgLoss.replace(0, 1e-10)
relativeStrength = avgGain / avgLoss
dfDaily['RSI_10'] = 100 - (100 / (1 + relativeStrength))

#add moving average convergence divergence (MACD) over 10-day and 30-day periods
macdShortTerm = dfDaily['Close'].ewm(span=10, adjust=False).mean()
macdLongTerm = dfDaily['Close'].ewm(span=30, adjust=False).mean()
dfDaily['MACD_10_30'] = macdShortTerm - macdLongTerm


#add Bollinger Bands over 20-day period
rollingMean = dfDaily['Close'].rolling(window=20, min_periods=1).mean()
rollingStd = dfDaily['Close'].rolling( window=20, min_periods=1).std()
dfDaily['Bollinger_Upper'] = rollingMean + (rollingStd * 2)
dfDaily['Bollinger_Lower'] = rollingMean - (rollingStd * 2)

#add On-Balance Volume (OBV)
dfDaily['OBV'] = (np.sign(dfDaily['Close'].diff()) * dfDaily['Volume']
                  ).fillna(0).cumsum()

#add rolling correlation between Close price and Volume over 10-day period
dfDaily['Close_Volume_Corr_10'] = dfDaily['Close'].rolling(
    window=10, min_periods=1).corr(dfDaily['Volume'])

#add atrioventricular (ATR) over 14-day period
dfDaily['High_Low'] = dfDaily['High'] - dfDaily['Low']
dfDaily['High_Close_Prev'] = np.abs(dfDaily['High'] - dfDaily['Close'].shift(1))
dfDaily['Low_Close_Prev'] = np.abs(dfDaily['Low'] - dfDaily['Close'].shift(1))
dfDaily['TR'] = dfDaily[['High_Low', 'High_Close_Prev', 'Low_Close_Prev']].max(axis=1)
dfDaily['ATR_14'] = dfDaily['TR'].rolling(window=14, min_periods=1).mean()

#add Fibonacci retracement levels
maxPrice = dfDaily['Close'].rolling(window=30, min_periods=1).max()
minPrice = dfDaily['Close'].rolling(window=30, min_periods=1).min()
priceRange = maxPrice - minPrice
dfDaily['Fibonacci_23.6'] = maxPrice - (priceRange * 0.236)
dfDaily['Fibonacci_38.2'] = maxPrice - (priceRange * 0.382)
dfDaily['Fibonacci_50.0'] = maxPrice - (priceRange * 0.5)
dfDaily['Fibonacci_61.8'] = maxPrice - (priceRange * 0.618)
dfDaily['Fibonacci_78.6'] = maxPrice - (priceRange * 0.786)

#add Williams %R over 14-day period
highestHigh = dfDaily['High'].rolling(window=14, min_periods=1).max()
lowestLow = dfDaily['Low'].rolling(window=14, min_periods=1).min()
dfDaily['Williams_%R_14'] = -100 * ((highestHigh - dfDaily['Close']) / (highestHigh - lowestLow))

#add Chaikin Money Flow (CMF) over 20-day period
moneyFlowMultiplier = ((dfDaily['Close'] - dfDaily['Low']) - (dfDaily['High'] - dfDaily['Close'])) / (dfDaily['High'] - dfDaily['Low'])
moneyFlowVolume = moneyFlowMultiplier * dfDaily['Volume']
dfDaily['CMF_20'] = moneyFlowVolume.rolling(window=20, min_periods=1).sum() / dfDaily['Volume'].rolling(window=20, min_periods=1).sum()

#normalize volume using mean and standard deviation
dfDaily['Volume_Normalized'] = (
    dfDaily['Volume'] - dfDaily['Volume'].mean()) / dfDaily['Volume'].std()

#cyclical encoding of day of week and time feature
dfDaily['Day_of_Week'] = dfDaily.index.dayofweek
dfDaily['Day_of_Month'] = dfDaily.index.day

dfDaily['Day_of_Week_Sin'] = np.sin(2 * np.pi * dfDaily['Day_of_Week'] / 7)
dfDaily['Day_of_Week_Cos'] = np.cos(2 * np.pi * dfDaily['Day_of_Week'] / 7)

#drop row with NaN values produced by feature engineering
dfDaily = dfDaily.dropna(how='any')

#show final dataframe
dfDaily.head()


,Open,High,Low,Close,Volume,TimeDiff,Return,Moving_Average_7,Moving_Average_30,Volatility_7,...,Fibonacci_50.0,Fibonacci_61.8,Fibonacci_78.6,Williams_%R_14,CMF_20,Volume_Normalized,Day_of_Week,Day_of_Month,Day_of_Week_Sin,Day_of_Week_Cos
2012-01-02 00:00:00+00:00,4.84,5.00,4.84,5.00,10.100000,86400.0,0.033058,4.920000,4.920000,0.113137,...,4.920,4.90112,4.87424,-0.000000,1.000000,-0.832109,0,2,0.000000,1.000000
2012-01-03 00:00:00+00:00,5.00,5.32,5.00,5.29,107.085281,86400.0,0.058000,5.043333,5.043333,0.228108,...,5.065,5.01190,4.93630,-4.054054,0.842132,-0.821501,1,3,0.781831,0.623490
2012-01-04 00:00:00+00:00,5.29,5.57,4.93,5.57,107.233260,86400.0,0.052930,5.175000,5.175000,0.322542,...,5.205,5.11886,4.99622,-0.000000,0.914348,-0.821485,2,4,0.974928,-0.222521
2012-01-05 00:00:00+00:00,5.57,6.46,5.57,6.42,70.328742,86400.0,0.152603,5.424000,5.424000,0.622921,...,5.630,5.44356,5.17812,-2.127660,0.913370,-0.825521,3,5,0.433884,-0.900969
2012-01-06 00:00:00+00:00,6.42,6.90,6.40,6.40,55.855834,86400.0,-0.003115,5.586667,5.586667,0.684972,...,5.630,5.44356,5.17812,-21.551724,0.616998,-0.827104,4,6,-0.433884,-0.900969


In [ ]:
#check missing values again
missingValues = dfDaily.isna().sum()
print("Missing Values in each column after feature engineering:\n",
      missingValues)

Missing Values in each column after feature engineering:
 Open                    0
High                    0
Low                     0
Close                   0
Volume                  0
TimeDiff                0
Return                  0
Moving_Average_7        0
Moving_Average_30       0
Volatility_7            0
Volatility_30           0
Cumulative_Return       0
RSI_10                  0
MACD_10_30              0
Bollinger_Upper         0
Bollinger_Lower         0
OBV                     0
Close_Volume_Corr_10    0
High_Low                0
High_Close_Prev         0
Low_Close_Prev          0
TR                      0
ATR_14                  0
Fibonacci_23.6          0
Fibonacci_38.2          0
Fibonacci_50.0          0
Fibonacci_61.8          0
Fibonacci_78.6          0
Williams_%R_14          0
CMF_20                  0
Volume_Normalized       0
Day_of_Week             0
Day_of_Month            0
Day_of_Week_Sin         0
Day_of_Week_Cos         0
dtype: int64


In [ ]:
dfDaily.tail()

,Open,High,Low,Close,Volume,TimeDiff,Return,Moving_Average_7,Moving_Average_30,Volatility_7,...,Fibonacci_50.0,Fibonacci_61.8,Fibonacci_78.6,Williams_%R_14,CMF_20,Volume_Normalized,Day_of_Week,Day_of_Month,Day_of_Week_Sin,Day_of_Week_Cos
2025-04-05 00:00:00+00:00,75542.152318,75543.569536,74434.596026,75405.741722,517.402365,86400.0,-0.000005,75406.854305,75411.119205,0.801149,...,75411.119205,75409.850119,75408.043285,-12.719773,0.752914,-0.776624,5,5,-0.974928,-0.222521
2025-04-06 00:00:00+00:00,75542.576159,75543.284768,74434.298013,75405.370861,513.594937,86400.0,-0.000005,75406.483444,75410.748344,0.801149,...,75410.748344,75409.479258,75407.672424,-12.727359,0.752762,-0.777040,6,6,-0.781831,0.623490
2025-04-07 00:00:00+00:00,75543.000000,75543.000000,74434.000000,75405.000000,509.787509,86400.0,-0.000005,75406.112583,75410.377483,0.801149,...,75410.377483,75409.108397,75407.301563,-12.734945,0.752609,-0.777457,0,7,0.000000,1.000000
2025-04-08 00:00:00+00:00,75556.500000,75641.500000,74511.000000,75384.500000,380.761161,86400.0,-0.000272,75402.866131,75409.335651,8.123431,...,75399.942053,75396.297728,75391.109199,-21.283644,0.745169,-0.791569,1,8,0.781831,0.623490
2025-04-09 00:00:00+00:00,75570.000000,75740.000000,74588.000000,75364.000000,251.734813,86400.0,-0.000272,75396.744087,75407.622848,16.473491,...,75389.506623,75383.487060,75374.916834,-28.790199,0.735044,-0.805680,2,9,0.974928,-0.222521


### Form Dataset

In [ ]:
#extract features and target variable
features = dfDaily.drop(columns=['Close'])
target = dfDaily['Close']

#create Dataset
xValues = features.values
yValues = target.values

#set window size for time series forecasting
windowSize = 30

#use sliding window to create sequences
xWindows = []
yWindows = []
for i in range(len(xValues) - windowSize):
    xWindows.append(xValues[i:i+windowSize])
    yWindows.append(yValues[i+windowSize])

xWindows = np.array(xWindows)
yWindows = np.array(yWindows)

print("Feature windows shape:", xWindows.shape)
print("Target windows shape:", yWindows.shape)

Feature windows shape: (4817, 30, 34)
Target windows shape: (4817,)


### Split Data for Training, Validation and Testing

In [ ]:
#split data into training, validation, and testing sets
totalSamples = xWindows.shape[0]
trainSize = int(totalSamples * 0.8)
validationSize = int(totalSamples * 0.1)
testSize = totalSamples - trainSize - validationSize

trainX = xWindows[:trainSize]
trainY = yWindows[:trainSize]

validationX = xWindows[trainSize:trainSize + validationSize]
validationY = yWindows[trainSize:trainSize + validationSize]

testX = xWindows[trainSize + validationSize:]
testY = yWindows[trainSize + validationSize:]

print("Training set shape:", trainX.shape, trainY.shape)
print("Validation set shape:", validationX.shape, validationY.shape)
print("Testing set shape:", testX.shape, testY.shape)

Training set shape: (3853, 30, 34) (3853,)
Validation set shape: (481, 30, 34) (481,)
Testing set shape: (483, 30, 34) (483,)


In [ ]:
#normalize features
scaler =  StandardScaler()
numFeatures = trainX.shape[2]

#reshape for scaling
trainXTemp = trainX.reshape(-1, numFeatures)
validationXTemp = validationX.reshape(-1, numFeatures)
testXTemp = testX.reshape(-1, numFeatures)

#apply scaling
trainXTemp = scaler.fit_transform(trainXTemp)
validationXTemp = scaler.transform(validationXTemp)
testXTemp = scaler.transform(testXTemp)

#reshape back to original shape
trainX = trainXTemp.reshape(trainX.shape)
validationX = validationXTemp.reshape(validationX.shape)
testX = testXTemp.reshape(testX.shape)

#scale target variable
targetScaler = StandardScaler()
trainY = targetScaler.fit_transform(trainY.reshape(-1, 1)).flatten()
validationY = targetScaler.transform(validationY.reshape(-1, 1)).flatten()
testY =targetScaler.transform(testY.reshape(-1, 1)).flatten()

Note: fit_transform is not called on validation and test datasets because that would leak information causing bias in our model resulting in inflated performance.

### Store Windowed Data into Tensors

In [ ]:
#convert to tensors
trainXTensor = torch.tensor(trainX, dtype=torch.float32)
trainYTensor = torch.tensor(trainY, dtype=torch.float32)

validationXTensor = torch.tensor(validationX, dtype=torch.float32)
validationYTensor = torch.tensor(validationY, dtype=torch.float32)


testXTensor = torch.tensor(testX, dtype=torch.float32)
testYTensor = torch.tensor(testY, dtype=torch.float32)

#create TensorDatasets
trainDataset = TensorDataset(trainXTensor,trainYTensor)
validationDataset = TensorDataset(validationXTensor, validationYTensor)
testDataset = TensorDataset(testXTensor, testYTensor)

#data loaders
batchSize = 32
trainLoader = DataLoader(trainDataset,batch_size=batchSize, shuffle=False)
validationLoader = DataLoader(
    validationDataset, batch_size=batchSize, shuffle=False)
testLoader = DataLoader(testDataset, batch_size=batchSize, shuffle=False)

#print dataset sizes
print("Number of training samples:", len(trainDataset))
print("Number of validation samples:", len(validationDataset))
print("Number of testing samples:", len(testDataset))

Number of training samples: 3853
Number of validation samples: 481
Number of testing samples: 483
